# KRR FINAL PROJECT GIACOMO COLOSIO

**Installation and import of the libraries** 

AS first thing I install and import the following libraries wich permit me use python to generate an RDF knowledge graph.

In [139]:
#requirements
!pip install -q rdflib sparqlwrapper owlrl gdown pandas

#import the libraries which i will use for the project
import owlrl
import gdown
import os
import pandas as pd
import rdflib
from SPARQLWrapper import SPARQLWrapper

**Generation of the directory and download of the data**

In [140]:
# create directory for data
if not os.path.exists("./data/"):
    os.mkdir("./data/")

# Download data
gdown.download(id="1UILSMenvqzFMsIhV6l4ZV1opJpRQZnwC", output="./data/rym_top_5000_all_time.csv", quiet=True)
gdown.download(id="1UNJ2iDX-xoWIekSA0ZwoqNOk3eAZjqrp", output="./data/albumlist.csv", quiet=True)

'./data/albumlist.csv'

This two CVS files will be the base for my resource description framework knowledge graph, which i will generate thanks to RDFlib.

### **Part 1: *Load the CSV files in two pandas DataFrames and join them***

Using pandas i upload the cvs file as two Dataframes, so two-dimensional, size-mutable, potentially heterogeneous tabular data, or in more simpler word a table composed by rows and columns wich i will use to construct my RDF knowledge graph. 
So using pd.read_csv(csvfile.csv) i will generate df1 (dataframe 1) and df2(data frame 2).

In [141]:
#generate df1
df1 = pd.read_csv('data/albumlist.csv', encoding='mac_roman')

Some characteristich of df1:

In [142]:
#i will not use pd.dataframe.info but i could use it to obatin similar information
print('df1 has as columns: \n')
n=0
for x in df1.columns:
  print(x, 'which types is', df1.dtypes[n])
  n=n+1
print()
print('the number of element of each columns are: ', df1.index.stop)

df1 has as columns: 

Number which types is int64
Year which types is int64
Album which types is object
Artist which types is object
Genre which types is object
Subgenre which types is object

the number of element of each columns are:  500


To have an idea of the structure of the table wich we need to use using the method pandas.DataFrame.head(n) it's possible to show as a table the firstn-rows, the default value of n is 5.

In [143]:
df1.head()

,Number,Year,Album,Artist,Genre,Subgenre
0,1,1967,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,Rock,"Rock & Roll, Psychedelic Rock"
1,2,1966,Pet Sounds,The Beach Boys,Rock,"Pop Rock, Psychedelic Rock"
2,3,1966,Revolver,The Beatles,Rock,"Psychedelic Rock, Pop Rock"
3,4,1965,Highway 61 Revisited,Bob Dylan,Rock,"Folk Rock, Blues Rock"
4,5,1965,Rubber Soul,The Beatles,"Rock, Pop",Pop Rock


In [144]:
df2 = pd.read_csv('data/rym_top_5000_all_time.csv')

Some characteristich of df2:

In [145]:
print('df2 has as columns:\n')
n=0
for x in df2.columns:
  print(x, 'which types is', df2.dtypes[n])
  n=n+1
print()
print('the number of element of each columns are: ', df2.index.stop)

df2 has as columns:

Ranking which types is float64
Album which types is object
Artist Name which types is object
Release Date which types is object
Genres which types is object
Descriptors which types is object
Average Rating which types is float64
Number of Ratings which types is object
Number of Reviews which types is int64

the number of element of each columns are:  5000


As before to have an idea of the structure of the table wich we need to use using the method pandas.DataFrame.head(n) it's possible to show as a table the firstn-rows, the default value of n is 5.

In [146]:
df2.head()

,Ranking,Album,Artist Name,Release Date,Genres,Descriptors,Average Rating,Number of Ratings,Number of Reviews
0,1.0,OK Computer,Radiohead,16 June 1997,"Alternative Rock, Art Rock","melancholic, anxious, futuristic, alienation, ...",4.23,"70,382",1531
1,2.0,Wish You Were Here,Pink Floyd,12 September 1975,"Progressive Rock, Art Rock","melancholic, atmospheric, progressive, male vo...",4.29,"48,662",983
2,3.0,In the Court of the Crimson King,King Crimson,10 October 1969,"Progressive Rock, Art Rock","fantasy, epic, progressive, philosophical, com...",4.30,"44,943",870
3,4.0,Kid A,Radiohead,3 October 2000,"Art Rock, Experimental Rock, Electronic","cold, melancholic, futuristic, atmospheric, an...",4.21,"58,590",734
4,5.0,To Pimp a Butterfly,Kendrick Lamar,15 March 2015,"Conscious Hip Hop, West Coast Hip Hop, Jazz Rap","political, conscious, poetic, protest, concept...",4.27,"44,206",379


**Join dataframes**

To join the dataframe i need to generate a column which we call [ALBUMminuscuolo] wich contain the album with only lowercase letter, with this we could generate new dataframe which contains all the infomation of the two dataframes of a singule album. 

In [147]:
#generate new column
df1['ALBUMminuscolo'] = df1.Album.str.lower()
df1.head()

,Number,Year,Album,Artist,Genre,Subgenre,ALBUMminuscolo
0,1,1967,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,Rock,"Rock & Roll, Psychedelic Rock",sgt. pepper's lonely hearts club band
1,2,1966,Pet Sounds,The Beach Boys,Rock,"Pop Rock, Psychedelic Rock",pet sounds
2,3,1966,Revolver,The Beatles,Rock,"Psychedelic Rock, Pop Rock",revolver
3,4,1965,Highway 61 Revisited,Bob Dylan,Rock,"Folk Rock, Blues Rock",highway 61 revisited
4,5,1965,Rubber Soul,The Beatles,"Rock, Pop",Pop Rock,rubber soul


In [148]:
df2['ALBUMminuscolo'] = df2.Album.str.lower()
df2.head()

,Ranking,Album,Artist Name,Release Date,Genres,Descriptors,Average Rating,Number of Ratings,Number of Reviews,ALBUMminuscolo
0,1.0,OK Computer,Radiohead,16 June 1997,"Alternative Rock, Art Rock","melancholic, anxious, futuristic, alienation, ...",4.23,"70,382",1531,ok computer
1,2.0,Wish You Were Here,Pink Floyd,12 September 1975,"Progressive Rock, Art Rock","melancholic, atmospheric, progressive, male vo...",4.29,"48,662",983,wish you were here
2,3.0,In the Court of the Crimson King,King Crimson,10 October 1969,"Progressive Rock, Art Rock","fantasy, epic, progressive, philosophical, com...",4.30,"44,943",870,in the court of the crimson king
3,4.0,Kid A,Radiohead,3 October 2000,"Art Rock, Experimental Rock, Electronic","cold, melancholic, futuristic, atmospheric, an...",4.21,"58,590",734,kid a
4,5.0,To Pimp a Butterfly,Kendrick Lamar,15 March 2015,"Conscious Hip Hop, West Coast Hip Hop, Jazz Rap","political, conscious, poetic, protest, concept...",4.27,"44,206",379,to pimp a butterfly


Now we join the two dataframes, to do it we could use two different methods: 

1.left.join(right, on=key_or_keys)

2.pd.merge(
    left, right, left_on=key_or_keys, right_index=True, how="left", sort=False
)
These two function calls are completely equivalent, in my project i have decide to use the second method and i have obtain the following final dataframe wich i called df:

In [149]:
#generate the final dataframe df
df=pd.merge(df1, df2, on='ALBUMminuscolo',how='inner')
df.rename(columns = {'Album_x':'Album', 'Album_y':'ALBUMNAME'}, inplace = True)

This merge of df1 and df2 gives the dataframe df generated by the merge of the two previous dataframes. 
Now i print some df informations:

In [150]:
print('df has as columns: \n')
n=0
for x in df.columns:
  print(x, 'which types is', df.dtypes[n])
  n=n+1
print()
print('the number of element of each columns are: ', len(df))

df has as columns: 

Number which types is int64
Year which types is int64
Album which types is object
Artist which types is object
Genre which types is object
Subgenre which types is object
ALBUMminuscolo which types is object
Ranking which types is float64
ALBUMNAME which types is object
Artist Name which types is object
Release Date which types is object
Genres which types is object
Descriptors which types is object
Average Rating which types is float64
Number of Ratings which types is object
Number of Reviews which types is int64

the number of element of each columns are:  342


**How i manipulate data of the graph?**

I can notice that the column Albuminuscolo of df1 and the same column for the df2 was the based for this merging, in fact this new dataframe df contains the rows of the two previous dataframes wich have the same album. Seeing the two initial dataframes we could observe that some columns probably could contains similar infotmation.
This means that this merged dataframe contain some similar information in its comuln, for example:
1. [ALBUMNAME] from df2 and [Album] from df1.

> *  In this case i decide to use only the [Album] column.

1. [Artist Name] from df2 and [Artist] from df1.




> * In this case i decide to use only the [Artist] column.




2. [Genre] and [Subgenere] from df1 and [Genres] from df2


>*   Here i decide to consider a only [Genre] and [Subgenres].


### **PART 2: *Create an `RDFLib Graph` and add triples from the data of the dataframes***
Using the RDFlib library I generate an RDF graph in wich i will add the information containing in df, the merge of the two inizial dataframes.

In [151]:
# imports will be explained when they are used
from rdflib import Graph, URIRef, BNode, Literal, FOAF, RDF, RDFS, XSD

#create the new rdf graph using rdflib
rdf_graph=Graph() 

Now before to add the RDF-triple i will define the namespace ex, wich for the moment i use to construct the graph, in this project i will use:
- rdf_lib which permit me to create a graph, to generate triple, add a domain and a range to properties, generate a taxonomies , add rdfs inference and other thing wich i show later.
- owlrl which simulates the OWL 2 (Web ontology lenguage) thanks to this i will generate all the inverse property whic i need, i will classificate the properties in object and data properties, i wil add some OWL inference and at the end i will connect this graph with  DBPEDIA and it's components(e.g. entities, object property, dataproperty)

In [152]:
from rdflib import Namespace
ex = Namespace('http://example.org/')
rdf_graph.bind('ex', ex)

Thanks to the dataframe(df) i will generate the triples wich construct the initial rdf graph.
For the df i generate the following relations:

As object Properties:
1.  The relation which connects each [Album] to its [Artist] and it's inverse.

4. The relation which connectswhich connects each [Album] to each [Genre] and it's inverse.
5. The relation which connectswhich connects each [Album] to each [Subgenre] and it's inverse.

Then for all of this propierties i will specifies with owlrl that they are object properties using the following triple: 
- `ex:[property] a owl:ObjectProperty .`

as data Property:
1. The which connects each [Album] to its Name which is a string value.
1. The which connects each [Album] to its [Year] which is an integer 
value.
3. The which connects each [Album] to its [Release_Date] wich is a string value.
2.  The which connects each [Album] to its [Ranking] which is an float value.
3.  The which connects each [Album] to its [Descriptors] which is string value.
4.  The which connects each [Album] to its [Average Rating] which is float value.
5.  The which connects each [Album] to its [Number of ratings] which is string value.
6. The which connects each [Album] to its [Number of views] which is an integer value.

Obviusly it's possible to generate also other properties but the fundamental one are contained there, such as the relation wich connect every [Artist] to its own [Album].


**Object Propierties**



*   Start adding the triple wich connect [Album] to its [Artist] thanks to the object properies: *Album_Has_Artist*.

To do it we need to sobstitute in the columns [Album] and [Artist] all the ' ' with '_' to generate valid Uris, for the same reason we need to delete all the special characters wich invalidate the URI.






In [153]:
from owlrl import OWL #import owl to add triple 
#Add in the graph all the triples wich connect an [Album] to its [Artist], use ex.Album_Has_Artist property
for inded, row in df.iterrows():
   Album=row['Album']
   Album=Album.replace(' ','_')
   Album=Album.replace('"','')
   Album=Album.replace(' ','_')
   Artist=row['Artist']
   Artist=Artist.replace(' ','_')
   #Add the triple 
   rdf_graph.add((URIRef(ex+str(Album)), URIRef(ex.Album_Has_Artist), URIRef(ex+Artist)))
rdf_graph.add((URIRef(ex.Album_Has_Artist), RDF.type ,OWL.ObjectProperty))#define predicate as obj properties
rdf_graph.add((URIRef(ex.Artist_Composed_Album), RDF.type ,OWL.ObjectProperty))
rdf_graph.add((URIRef(ex.Artist_Composed_Album), OWL.inverseOf ,(URIRef(ex.Album_Has_Artist))))

<Graph identifier=N2695d8658db840ae9911a7798a1006b7 (<class 'rdflib.graph.Graph'>)>

*   Now i add the triple wich connect [Album] to its [Genre] thanks to the object properies: *Album_Has_Genre*.

To do it we need the same of sobstitution of before but we also do another thing. In fact an Album could have 2 or more differents genres and so we need to generate different URI for all the possible genres contain in [Genre].
Obvioulsy the text need some modification for the URI generation.

In [154]:
#Add in the graph all the triples wich connect an [Album] to its [Genre], use ex.Album_Has_Genre property
for inded, row in df.iterrows():
   Album=row['Album']
   Album=Album.replace(' ','_')
   Album=Album.replace('"','')
   Album=Album.replace(' ','_')
   txt=row['Genre']
   genres = txt.split(',')
   for x in genres:
      if len(x)>1:
        u=x.replace(' ','') 
        q=u.replace('&','/')
        z=q.replace(' ','')
        if '/' in z:
          z1=z.split('/')
          for y in z1:
            if len(y)>1: 
              rdf_graph.add((URIRef(ex+str(Album)), URIRef(ex.Album_Has_Genre), URIRef(ex+y)))
        else:
          rdf_graph.add((URIRef(ex+str(Album)), URIRef(ex.Album_Has_Genre), URIRef(ex+z)))
rdf_graph.add((URIRef(ex.Album_Has_Genre), RDF.type ,OWL.ObjectProperty))
rdf_graph.add((URIRef(ex.Genre_Of_Album), RDF.type ,OWL.ObjectProperty))
rdf_graph.add((URIRef(ex.Genre_Of_Album), OWL.inverseOf ,(URIRef(ex.Album_Has_Genre))))

<Graph identifier=N2695d8658db840ae9911a7798a1006b7 (<class 'rdflib.graph.Graph'>)>

*   Now i add the triple wich connect [Album] to its [SubGenre] thanks to the object properies: *Album_Has_Subgenres*.

To do it we need the same of sobstitution of before but we also do another thing. In fact an Album could have 2 or more different subgenres and so we need it to generate different URI for all the possible subgenres contain in [Subgenre].
Obvioulsy the text need some modification for the URI generation.

Sometimes the genres is none, if it's none i dont' add the triple obvioulsy.

In [155]:
#Add in the graph all the triples wich connect an [Album] to its [Subgenre], use ex.Album_Has_Subgenre property
for inded, row in df.iterrows():
   Album=row['Album']
   Album=Album.replace(' ','_')
   Album=Album.replace('"','')
   Album=Album.replace(' ','_')
   txt=row['Subgenre']
   subgenre= txt.split(',')
   for x in subgenre:
      if len(x)>1:
        z=x.replace(' ','')
        if z != 'None':
          #print(z)
          rdf_graph.add((URIRef(ex+str(Album)), URIRef(ex.Album_Has_Subgenre), URIRef(ex+z)))#Add the triple
rdf_graph.add((URIRef(ex.Album_Has_Subgenre), RDF.type ,OWL.ObjectProperty))
rdf_graph.add((URIRef(ex.Album_Has_Subgenre), RDF.type ,OWL.ObjectProperty))
rdf_graph.add((URIRef(ex.Subgenre_Of_Album), RDF.type ,OWL.ObjectProperty))
rdf_graph.add((URIRef(ex.Subgenre_Of_Album), OWL.inverseOf ,(URIRef(ex.Album_Has_Subgenre))))

<Graph identifier=N2695d8658db840ae9911a7798a1006b7 (<class 'rdflib.graph.Graph'>)>

**Data propierties**

Here I need to generate the relation between an object and a data such as an integer or a string, to do it with the namespace i genrate the propierties wich connect an URI to a literal.
I also define all the dataproperty as OWL:DatatypeProperty.
- Now i start adding the Relation betwenn the URI of the [Album] and it's name(string)

In [156]:
#Add in the graph all the triples wich connect an [Album] to its name, use ex.AlbumName property
for inded, row in df.iterrows():
   Album=row['Album']
   Album=Album.replace(' ','_')
   Album=Album.replace('"','')
   Album=Album.replace(' ','_')
   
   #Add the triple 
   rdf_graph.add((URIRef(ex+str(Album)), URIRef(ex.AlbumName), Literal(row['Album'], datatype=XSD.string)))
#specify that album name is a property
rdf_graph.add((URIRef(ex.AlbumName), RDF.type , OWL.DatatypeProperty))


<Graph identifier=N2695d8658db840ae9911a7798a1006b7 (<class 'rdflib.graph.Graph'>)>

- Now i add the Relation *between* the URI of the [Album] and it's [Release Date] wich is a string.

In [157]:
#Add in the graph all the triples wich connect an [Album] to its release date, use ex.AlbumReleaseDate property
for inded, row in df.iterrows():
   Album=row['Album']
   Album=Album.replace(' ','_')
   Album=Album.replace('"','')
   Album=Album.replace(' ','_')
   
   #Add the triple 
   rdf_graph.add((URIRef(ex+str(Album)), URIRef(ex.AlbumReleaseDate), Literal(row['Release Date'], datatype=XSD.string)))
   #print(rdf_graph.serialize())
rdf_graph.add((URIRef(ex.AlbumReleaseDate), RDF.type , OWL.DatatypeProperty))

<Graph identifier=N2695d8658db840ae9911a7798a1006b7 (<class 'rdflib.graph.Graph'>)>

- Now i  add the Relation between the URI of the [Album] and it's [Year] wich is an integer.

In [158]:
#Add in the graph all the triples wich connect an [Album] to its year, use ex.AlbumYear property
for inded, row in df.iterrows():
   Album=row['Album']
   Album=Album.replace(' ','_')
   Album=Album.replace('"','')
   Album=Album.replace(' ','_')
   
   #Add the triple 
   rdf_graph.add((URIRef(ex+str(Album)), URIRef(ex.AlbumYear), Literal(row['Year'], datatype=XSD.int)))
#print(rdf_graph.serialize())
rdf_graph.add((URIRef(ex.AlbumYear), RDF.type , OWL.DatatypeProperty))

<Graph identifier=N2695d8658db840ae9911a7798a1006b7 (<class 'rdflib.graph.Graph'>)>

- Now i start add the Relation between the URI of the [Album] and it's [Ranking] wich is a float.

In [159]:
#Add in the graph all the triples wich connect an [Album] to its ranking, use ex.AlbumRanking property
for inded, row in df.iterrows():
   Album=row['Album']
   Album=Album.replace(' ','_')
   Album=Album.replace('"','')
   Album=Album.replace(' ','_')
   
   #Add the triple 
   rdf_graph.add((URIRef(ex+str(Album)), URIRef(ex.AlbumRanking), Literal(row['Ranking'], datatype=XSD.float)))
rdf_graph.add((URIRef(ex.AlbumRanking), RDF.type , OWL.DatatypeProperty))   

<Graph identifier=N2695d8658db840ae9911a7798a1006b7 (<class 'rdflib.graph.Graph'>)>

- Now i add the Relation betwenn the URI of the [Album] and it's [Descriptors] wich is a list of string value.

In [160]:
#Add in the graph all the triples wich connect an [Album] to its [Descriptors], use ex.AlbumDescriptor property
for inded, row in df.iterrows():
   Album=row['Album']
   Album=Album.replace(' ','_')
   Album=Album.replace('"','')
   Album=Album.replace(' ','_')
   txt=row['Descriptors']
   subdescriptors= txt.split(',')
   for x in subdescriptors:
      if len(x)>1:
        z=x.replace(' ','')
      
       
        #Add the triple 
        rdf_graph.add((URIRef(ex+str(Album)), URIRef(ex.AlbumDescriptor), URIRef(ex+z)))
   #print(rdf_graph.serialize())
rdf_graph.add((URIRef(ex.AlbumDescriptor), RDF.type , OWL.DatatypeProperty))  

<Graph identifier=N2695d8658db840ae9911a7798a1006b7 (<class 'rdflib.graph.Graph'>)>

- Now i add the Relation between the URI of the [Album] and it's [Average Rating] wich is a float.

In [161]:
#Add in the graph all the triples wich connect an [Album] to its [Average Rating], use ex.AlbumAverageRate property
for inded, row in df.iterrows():
   Album=row['Album']
   Album=Album.replace(' ','_')
   Album=Album.replace('"','')
   Album=Album.replace(' ','_')
   
   #Add the triple 
   rdf_graph.add((URIRef(ex+str(Album)), URIRef(ex.AlbumAverageRate), Literal(row['Average Rating'], datatype=XSD.float)))
   #print(rdf_graph.serialize())
rdf_graph.add((URIRef(ex.AlbumAverageRate), RDF.type , OWL.DatatypeProperty))  

<Graph identifier=N2695d8658db840ae9911a7798a1006b7 (<class 'rdflib.graph.Graph'>)>

- Now i  adding the Relation between the URI of the [Album] and it's [Number of Ratings] wich is a string.

In [162]:
#Add in the graph all the triples wich connect an [Album] to its [Number of Ratings], use ex.AlbumNumRatings property
for inded, row in df.iterrows():
   Album=row['Album']
   Album=Album.replace(' ','_')
   Album=Album.replace('"','')
   Album=Album.replace(' ','_')
   numrec=row['Number of Ratings']
   numrec1=numrec.replace(',','.')
   #Add the triple 
   rdf_graph.add((URIRef(ex+str(Album)), URIRef(ex.AlbumNumRatings), Literal(numrec1, datatype=XSD.int)))
#print(rdf_graph.serialize())
rdf_graph.add((URIRef(ex.AlbumNumRatings), RDF.type , OWL.DatatypeProperty))  

<Graph identifier=N2695d8658db840ae9911a7798a1006b7 (<class 'rdflib.graph.Graph'>)>

- Now i add the Relation between the URI of the [Album] and it's [Number of Views] wich is an integer.

In [163]:
#Add in the graph all the triples wich connect an [Album] to its [Number of Reviews], use ex.AlbumNumReviews property
for inded, row in df.iterrows():
   Album=row['Album']
   Album=Album.replace(' ','_')
   Album=Album.replace('"','')
   Album=Album.replace(' ','_')
   numrew=row['Number of Reviews']
   #Add the triple 
   rdf_graph.add((URIRef(ex+str(Album)), URIRef(ex.AlbumNumReviews),Literal(numrew, datatype=XSD.int)))
#print(rdf_graph.serialize())
rdf_graph.add((URIRef(ex.AlbumNumReviews), RDF.type , OWL.DatatypeProperty))  

<Graph identifier=N2695d8658db840ae9911a7798a1006b7 (<class 'rdflib.graph.Graph'>)>

### **3. Include a small ontology**

Now Using RDFlid i will add:

**Hierarchical class structures**
- rdfs:class and rdfs:subClassOf

**Domain and Range restrictions on properties**

 - rdfs:domain and rdfs:range

***Hierarchical class structures***

Here i define all the possible classes , so in a more clear view i define:
1. The class Album  and the i add all the value of [Album] column using inferences. 

2. The class Artist and then i add all the value of [Artist] column using inferences.

3. The class Genre and then i add all the values of [Genre] column using inferences.
4. The class Subgenres and then i add all the values of [subgenres] column using inferences.

- I start defining Album,Artist,genre and subgenre has classes 

In [164]:
rdf_graph.add((URIRef(ex.Album), (RDF.type) , (RDFS.Class)  ))
rdf_graph.add((URIRef(ex.Artist), (RDF.type) , (RDFS.Class)  ))
rdf_graph.add((URIRef(ex.Genre), (RDF.type) , (RDFS.Class)  ))
rdf_graph.add((URIRef(ex.Subgenre), (RDF.type)  , (RDFS.Class)  ))

<Graph identifier=N2695d8658db840ae9911a7798a1006b7 (<class 'rdflib.graph.Graph'>)>


**Domain and Range restrictions on properties**

 Here i define all the Doman and all the range of all possible ObjectPropierties and all the Domain of the Dataproperties, so in a more clear view i define:
1.  For the relation Album_Has_Artist we have as Domain [Album] and as range [Artist] and the inverse for it's inverse.
2. For the relation Album_Has_Genre we have as Domain [Album] and as range [Genre] and the inverse for it's inverse.

3. For the relation Album_Has_SubGenre we have as Domain [Album] and as range [Subgenre] and the inverse for it's inverse.


And for the dataproperties the Domain is always [Album] and for the range we have already insert the Literal with its specific datatypes.


- Start with the relation Album_Has_Artist

In [165]:
#Album_Has_Artist
rdf_graph.add((URIRef(ex.Album_Has_Artist),  (RDFS.domain), URIRef(ex.Album)))#Domain
rdf_graph.add((URIRef(ex.Album_Has_Artist),  (RDFS.range), URIRef(ex.Artist)))#Range

#iNVERSE in point 2 

<Graph identifier=N2695d8658db840ae9911a7798a1006b7 (<class 'rdflib.graph.Graph'>)>

- Now on the relation: Album_Has_Genre




In [166]:
rdf_graph.add((URIRef(ex.Album_Has_Genre),  (RDFS.domain), URIRef(ex.Album)))#Domain
rdf_graph.add((URIRef(ex.Album_Has_Genre),  (RDFS.range), URIRef(ex.Genre)))#Range

#INVERSE in point 2

<Graph identifier=N2695d8658db840ae9911a7798a1006b7 (<class 'rdflib.graph.Graph'>)>

- Now on the relation: Album_Has_Subgenre

In [167]:
rdf_graph.add((URIRef(ex.Album_Has_Subgenre),  (RDFS.domain), URIRef(ex.Album)))#Domain
rdf_graph.add((URIRef(ex.Album_Has_Subgenre),  (RDFS.range), URIRef(ex.Subgenre)))#Range

<Graph identifier=N2695d8658db840ae9911a7798a1006b7 (<class 'rdflib.graph.Graph'>)>

Now i add all the domain of the dataproperties:

In [168]:
rdf_graph.add((URIRef(ex.AlbumName),  (RDFS.domain), URIRef(ex.Album)))#Domain
rdf_graph.add((URIRef(ex.AlbumReleaseDate),  (RDFS.domain), URIRef(ex.Album)))#Domain
rdf_graph.add((URIRef(ex.Year),  (RDFS.domain), URIRef(ex.Album)))#Domain
rdf_graph.add((URIRef(ex.AlbumRanking),  (RDFS.domain), URIRef(ex.Album)))#Domain
rdf_graph.add((URIRef(ex.AlbumDescriptor),  (RDFS.domain), URIRef(ex.Album)))#Domain
rdf_graph.add((URIRef(ex.AlbumAverageRate),  (RDFS.domain), URIRef(ex.Album)))#Domain
rdf_graph.add((URIRef(ex.AlbumNumRatings),  (RDFS.domain), URIRef(ex.Album)))#Domain
rdf_graph.add((URIRef(ex.AlbumNumReviews),  (RDFS.domain), URIRef(ex.Album)))#Domain

<Graph identifier=N2695d8658db840ae9911a7798a1006b7 (<class 'rdflib.graph.Graph'>)>

### **4. Genres taxonomy**

In these section we add a particular taxonomies to my KB, in fact in the first dataframe has 'Genre' and 'Subgenre' columns. A taxonomy is a classification structure for objects of a given domain. Usually hierarchies
with tree-like structures, and in this particular case we a a mereology which consist in a part-of relation.

Now i extract a genres taxonomy from the dataset and will include it in my RDF KG. This will be needed later for inferring genres from subgenres.
To do it i will generate a classe for each genre,  and i instantiate the fact that all this classes are a subclasses of the class genre wich i have already created. Then i add for all the classes of genres the sublasses of subgenres. 

Then at the end we add triples to the graph like:
- `[album] rdf:type [genre]`.
- `[genre] rdf:type rdfs:Class`.
- `[genre] rdfs:subClassOf Genre`.
- `[subgenre] rdfs:subClassOf [genre]`.




Start now from the first type of triple, this triple are very similar and appears the same of [Album] ex:Has_Genre [Genre]:
So to add it i use the same strategies but with a different predicate (rdf:type)

In [169]:
#Add in the graph all the triples wich connect an [Album] to its [Genre], use rdf:type property
for inded, row in df.iterrows():
   Album=row['Album']
   Album=Album.replace(' ','_')
   Album=Album.replace('"','')
   Album=Album.replace(' ','_')
   txt=row['Genre']
   genres = txt.split(',')
   for x in genres:
      if len(x)>1:
        u=x.replace(' ','') 
        q=u.replace('&','/')
        z=q.replace(' ','')
        if '/' in z:
          z1=z.split('/')
          for y in z1:
            if len(y)>1: 
              rdf_graph.add((URIRef(ex+str(Album)), RDF.type, URIRef(ex+y)))
        else:
          rdf_graph.add((URIRef(ex+str(Album)), RDF.type, URIRef(ex+z)))

Now i generate all the triple of the types:
- `[genre] rdf:type rdfs:Class`.--> To generate a class for each genre
- `[genre] rdfs:subClassOf Genre`.--> To specify the mereology(part of relation)


In [170]:
#generate an empty list with all possible genre
l=[]
#Store all the possibile genre in a list
for inded, row in df.iterrows():
   txt=row['Genre']
   genres= txt.split(',')
   for x in genres:
      if len(x)>1:
        u=x.replace(' ','') 
        q=u.replace('&','/')
        z=q.replace(' ','')
        
        if '/' in z:
          z1=z.split('/')
          for y in z1:
            if len(y)>1:
              if y not in l:
                l.append(y)
        else:
          if x not in l:
            l.append(x)
#generate a class for all the element of the list
for genre in l:
  x=genre.replace(' ','')
  rdf_graph.add((URIRef(ex+x),(RDF.type),(RDFS.Class)))
  rdf_graph.add((URIRef(ex+x),(RDFS.subClassOf),URIRef(ex.Genre)))

Here i add the triple of the type:
- `[subgenre] rdfs:subClassOf [genre]`.

To do it i use the following strategies:
1. If an album have only 1 Genre (or in other words a unique value in the column [Genre] of the first dataframe)  i add to the KB the triple: `[subgenre] rdfs:subClassOf [genre]` for all the values(subgeneres) contained in the column [Subgeneres]. Obviuosly i will do it for all the rows of the final dataframe df.

2. In the case of two or more genres in the  [Genre] column i add for all this values the triple `[subgenre] rdfs:subClassOf [genre]` for all the values(subgeneres) contained in the column [Subgeneres]. 

In [171]:
#Add in the graph all the possible triple of the type [subgenre] rdfs:subClassOf [genre]
for inded, row in df.iterrows():
  txtg=row['Genre']
  txts=row['Subgenre']
  genres=txtg.split(',')
  subgenres = txts.split(',')
  for x in genres:
      if len(x)>1:
        u=x.replace(' ','') 
        q=u.replace('&','/')
        z=q.replace(' ','')
        if '/' in z:
          z1=z.split('/')
          for y in z1:
            if len(y)>1:
              for sub in subgenres:
                if len(sub)>1:
                  sub1=sub.replace(' ','')
                  if sub1!='None':
                    rdf_graph.add((URIRef(ex+sub1),(RDFS.subClassOf),URIRef(ex+y)))


In [172]:
#Print Graph
print(rdf_graph.serialize())

#rdf_graph.remove((None, None, None))  # if need remove all triples 

@prefix ex: <http://example.org/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:Album a rdfs:Class .

ex:Artist a rdfs:Class .

ex:Blues a rdfs:Class ;
    rdfs:subClassOf ex:Genre .

ex:Country a rdfs:Class ;
    rdfs:subClassOf ex:Country,
        ex:Funk,
        ex:Genre,
        ex:Soul .

ex:Electronic a rdfs:Class ;
    rdfs:subClassOf ex:Genre .

ex:Folk a rdfs:Class ;
    rdfs:subClassOf ex:Country,
        ex:Genre .

ex:Funk a rdfs:Class ;
    rdfs:subClassOf ex:Country,
        ex:Funk,
        ex:Genre,
        ex:Screen,
        ex:Soul,
        ex:Stage .

ex:Genre a rdfs:Class .

ex:HipHop a rdfs:Class ;
    rdfs:subClassOf ex:Funk,
        ex:Genre,
        ex:Soul .

ex:Jazz a rdfs:Class ;
    rdfs:subClassOf ex:Genre .

ex:Latin a rdfs:Class ;
    rdfs:subClassOf ex:Genre .

ex:Pop a rdfs:Class ;
    rdfs:subClassOf ex:Genre .

ex:Reggae a rdfs:Class ;
 

### **5. Load the RDF graph on a SPARQL Endpoint**
Usin
Now my RDF graph is finisched and i load it on my local SPARQL endpoint.
Then i will run a few SPARQL queries on the graph. To do it i will use SPARQLwrapper, a new library.

In [173]:
sparql = SPARQLWrapper("http://localhost:8890/sparql")
sparql.addDefaultGraph("ex:509653")
sparql.method = 'POST'


In [175]:
#using the rdf triple i add it on sparql
for s,p,o in rdf_graph:
    sparql.setQuery(f"INSERT DATA {{{s.n3()} {p.n3()} {o.n3()}.}}")
    sparql.query()

##### SPARL queries on my graph

The SPARQL Query Language can be used to formulate queries over RDF data ranging from simple graph pattern matching to complex queries. SPARQL queries contain a set of triple patterns called a basic graph pattern. Triple patterns are like RDF triples except that each of the subject, predicate and object may be a variable. Variables start with a question mark or a dollar sign.

A SPARQL query is formed by:
- Prologue, wich contains the prefix.
- Query form (Select/Ask/Construct/Describe)
- Query pattern
- Solution Modifiers

To make with  queries SPARQLwrapperi need to:
- set the query using sparql.setquery.
- set the format using sparql.setReturnFormat.
- decode the results.

Now i will do some queries using basic and more complex features.

**Query 1**: Set a query wich give as results set all the album and all the album name.

In [176]:
sparql.setQuery("prefix ex: <http://example.org/> select distinct * where {?x ex:AlbumName ?y}")

In [177]:
sparql.setReturnFormat("csv")

In [178]:
results = sparql.query()

results = results.convert()

print(results.decode())

"x","y"
"http://example.org/Sgt._Pepper's_Lonely_Hearts_Club_Band","Sgt. Pepper's Lonely Hearts Club Band"
"http://example.org/Psychocandy","Psychocandy"
"http://example.org/Los_Angeles","Los Angeles"
"http://example.org/Back_Stabbers","Back Stabbers"
"http://example.org/Rage_Against_the_Machine","Rage Against the Machine"
"http://example.org/Siamese_Dream","Siamese Dream"
"http://example.org/Live_Through_This","Live Through This"
"http://example.org/3_Feet_High_and_Rising","3 Feet High and Rising"
"http://example.org/Songs_in_the_Key_of_Life","Songs in the Key of Life"
"http://example.org/Pink_Flag","Pink Flag"
"http://example.org/Workingman's_Dead","Workingman's Dead"
"http://example.org/Surfer_Rosa","Surfer Rosa"
"http://example.org/Stand!","Stand!"
"http://example.org/Rain_Dogs","Rain Dogs"
"http://example.org/For_Your_Pleasure","For Your Pleasure"
"http://example.org/Abraxas","Abraxas"
"http://example.org/Bookends","Bookends"
"http://example.org/Graceland","Graceland"
"http://exam

 **Query 2**: Set a query wich give as a result all the rock.

In [179]:
sparql.setQuery("prefix ex: <http://example.org/> select distinct * where {?x a ex:Rock}")

In [180]:
sparql.setReturnFormat("csv")
results = sparql.query()
results = results.convert()
print(results.decode())

"x"
"http://example.org/(What's_the_Story)_Morning_Glory?"
"http://example.org/Let_It_Bleed"
"http://example.org/More_Songs_About_Buildings_and_Food"
"http://example.org/Astral_Weeks"
"http://example.org/Exile_in_Guyville"
"http://example.org/Master_of_Puppets"
"http://example.org/The_Velvet_Underground_&_Nico"
"http://example.org/Layla_and_Other_Assorted_Love_Songs"
"http://example.org/Blood_Sugar_Sex_Magik"
"http://example.org/Highway_to_Hell"
"http://example.org/Automatic_for_the_People"
"http://example.org/Paul_Simon"
"http://example.org/Help!"
"http://example.org/OK_Computer"
"http://example.org/Younger_Than_Yesterday"
"http://example.org/Wheels_of_Fire"
"http://example.org/Double_Nickels_on_the_Dime"
"http://example.org/Sgt._Pepper's_Lonely_Hearts_Club_Band"
"http://example.org/Tumbleweed_Connection"
"http://example.org/Van_Halen"
"http://example.org/Remain_in_Light"
"http://example.org/Quadrophenia"
"http://example.org/The_Paul_Butterfield_Blues_Band"
"http://example.org/Cheap_T

 **Query 3**: Set a query wich give as a result a top 10 of the album between the 1990 and 2010 in base of album ranking.

In [181]:
sparql.setQuery("""prefix ex: <http://example.org/> select distinct * where {?x ex:AlbumRanking ?y ; 
                    ex:AlbumYear ?z FILTER (?z > 1990 && ?z <2010) } Order by ?y limit 10""")

In [182]:
sparql.setReturnFormat("csv")
results = sparql.query()
results = results.convert()
print(results.decode())

"x","y","z"
"http://example.org/OK_Computer",1.0,1997
"http://example.org/Kid_A",4.0,2000
"http://example.org/Loveless",6.0,1991
"http://example.org/In_Rainbows",15.0,2007
"http://example.org/Illmatic",21.0,1994
"http://example.org/Dummy",42.0,1994
"http://example.org/The_Low_End_Theory",55.0,1991
"http://example.org/The_College_Dropout",68.0,2003
"http://example.org/Aquemini",84.0,1998
"http://example.org/Funeral",93.0,2004



**Query 4**: Set a query wich give as a result all the album of genre pop in with the lowest average rating with it's descriptors.

In [183]:
sparql.setQuery("prefix ex: <http://example.org/> select distinct * where {?x ex:Album_Has_Genre ex:Pop ; ex:AlbumAverageRate ?z  optional {?x ex:AlbumDescriptor ?q } } Order by ?z")

In [184]:
sparql.setReturnFormat("csv")
results = sparql.query()
results = results.convert()
print(results.decode())

"x","z","q"
"http://example.org/Bad",3.62,"http://example.org/malevocals"
"http://example.org/Bad",3.62,"http://example.org/energetic"
"http://example.org/Bad",3.62,"http://example.org/passionate"
"http://example.org/Bad",3.62,"http://example.org/love"
"http://example.org/Bad",3.62,"http://example.org/playful"
"http://example.org/Bad",3.62,"http://example.org/optimistic"
"http://example.org/Bad",3.62,"http://example.org/uplifting"
"http://example.org/Bad",3.62,"http://example.org/romantic"
"http://example.org/Bad",3.62,"http://example.org/rhythmic"
"http://example.org/Bad",3.62,"http://example.org/urban"
"http://example.org/Synchronicity",3.63,"http://example.org/malevocals"
"http://example.org/Synchronicity",3.63,"http://example.org/melancholic"
"http://example.org/Synchronicity",3.63,"http://example.org/dense"
"http://example.org/Synchronicity",3.63,"http://example.org/political"
"http://example.org/Synchronicity",3.63,"http://example.org/warm"
"http://example.org/Synchronicity",3.63

**Query 5**: Generate a query to ask if an EMINEM's album is in the graph.

In [185]:
sparql.setQuery("prefix ex: <http://example.org/> ask {?x ex:Album_Has_Artist ex:Eminem}")

In [186]:
sparql.setReturnFormat("csv")
results = sparql.query()
results = results.convert()
print(results.decode())

"bool"
1



### **6. Materialize inferences in the endpoint's graph**

Now i use SPARQL insert queries to materialize inferences that can be made from my KG.
The inference wich i can do are based in RDFS and OWL so i divide it into differents part, the first one foR RDFS inferences following the 13 rules, in the second one i use OWL inferences.

###### RDFS inferences.
Using the RDFS entailment rules i will say that:

**First thanks to the rules RDFS-2 and RDFAS-3 it's possible to say:**
- All the entities of the [Album] column are of type:Album.
- All the entities of the [Artist] column are of type:Artist.
- All the entities of the [Genre] column are of type:Genre.
- All the entities of the [SubGenre] column are of type:Subgenre.

This because for the property: Album_Has:Artist we need to have: as a domain an entity of type (so class) Album.
As a range an entity of type (so class) Artist. The same idea works for Album_Has_Genre and Album_Has_Subgenre.

To add inferences i nedd to use then query:
"insert {template} 
where{pattern}",  so the query add all the possible template triple which respect the where condition.

In [187]:
#ADD INFERENCES FOLLOWING THE RDFS RULE 2
sparql.setQuery("prefix ex: <http://example.org/> insert {?x rdf:type ex:Album} where {?x ex:Album_Has_Artist ?z }")

In [192]:
sparql.setReturnFormat("csv")
results = sparql.query()
results = results.convert()
#print(results.decode())

In [193]:
#ADD INFERENCE FOLLOWING THE RDFS RULE 3
sparql.setQuery("prefix ex: <http://example.org/> insert {?z rdf:type ex:Artist} where {?x ex:Album_Has_Artist ?z }") #Add resource type Artist

In [194]:
sparql.setReturnFormat("csv")
results = sparql.query()
results = results.convert()
#print(results.decode())

In [195]:
sparql.setQuery("prefix ex: <http://example.org/>  insert {?z rdf:type ex:Genre}  where {?x ex:Album_Has_Genre ?z }") #Add reources type Genre

In [196]:
sparql.setReturnFormat("csv")
results = sparql.query()
results = results.convert()
#print(results.decode())

In [197]:
sparql.setQuery("prefix ex: <http://example.org/>  insert {?z rdf:type ex:SubGenre}  where {?x ex:Album_Has_Subgenre ?z }")#Add resource type subgenre

In [198]:
sparql.setReturnFormat("csv")
results = sparql.query()
results = results.convert()
#print(results.decode())





E.g., if you have that `[album] rdf:type [genre1]` and `[genre1] rdfs:subClassOf [genre2]`, add the triple `[album] rdf:type [genre2]`. Same for domains, ranges, subclasses, subproperties, OWL features (if you use them), etc.

Run a few SPARQL queries on the graph.

**Now thanks to the rules RDFS4 it's possible to say:**
- All the subject of a triple are of the type resource.

In [199]:
sparql.setQuery("prefix ex: <http://example.org/>  insert {?x rdf:type rdfs:Resource}  where {?x ?y ?z}")
sparql.setReturnFormat("csv")
results = sparql.query()
results = results.convert()

**Now thanks to the rules RDFS5 it's possible to say:**
- if a property x is rdfs:subPropertyOf y and y is a rdfs:subPropertyOf z then	x is rdfs:subPropertyOf z

In [200]:
sparql.setQuery("prefix ex: <http://example.org/>  insert {?x rdfs:subClassOf ?z}  where {?x rdfs:subClassOf ?y.  ?y rdfs:subClassOf ?z }")
sparql.setReturnFormat("csv")
results = sparql.query()
results = results.convert()

**Now thanks to the rules RDFS7 it's possible to say:**
-  if A is a subPropertyOf B. and X A Y then X B Y .

In [201]:
sparql.setQuery("prefix ex: <http://example.org/>  insert {?x ?b ?y}  where {?a rdfs:subPropertyOf ?b.  ?x ?a ?y }")
sparql.setReturnFormat("csv")
results = sparql.query()
results = results.convert()

**Now thanks to the rules RDFS9 it's possible to say:**
-  if X rdfs:subClassOf Y and Z rdf:type X then	Z rdf:type Y .

In [202]:
sparql.setQuery("prefix ex: <http://example.org/>  insert {?z rdf:type ?y}  where {?z rdf:type ?x.  ?x rdfs:subClassOf ?y }")
sparql.setReturnFormat("csv")
results = sparql.query()
results = results.convert()

**Now thanks to the rules RDFS11 it's possible to say:**
-  If x rdfs:subClassOf y and  y rdfs:subClassOf z  then x rdfs:subClassOf z .

In [203]:
sparql.setQuery("prefix ex: <http://example.org/>  insert {?x rdfs:subClassOf ?z}  where {?y rdfs:subClassOf ?z.  ?x rdfs:subClassOf ?y  }")
sparql.setReturnFormat("csv")
results = sparql.query()
results = results.convert()

After this inference i make some queries:

In [204]:
#return all entities of type artist
sparql.setQuery("prefix ex: <http://example.org/> select distinct *  where {?x  a ex:Artist }")
sparql.setReturnFormat("csv")
results = sparql.query()
results = results.convert()
print(results.decode())

"x"
"http://example.org/Oasis"
"http://example.org/Paul_Simon"
"http://example.org/Van_Halen"
"http://example.org/The_Paul_Butterfield_Blues_Band"
"http://example.org/My_Bloody_Valentine"
"http://example.org/Rage_Against_the_Machine"
"http://example.org/DEVO"
"http://example.org/John_Prine"
"http://example.org/The_Beatles"
"http://example.org/Pretenders"
"http://example.org/Soundgarden"
"http://example.org/Black_Sabbath"
"http://example.org/Tracy_Chapman"
"http://example.org/George_Michael"
"http://example.org/Madonna"
"http://example.org/The_Doors"
"http://example.org/The_Velvet_Underground"
"http://example.org/David_Bowie"
"http://example.org/Radiohead"
"http://example.org/Crosby,_Stills_&_Nash"
"http://example.org/Ramones"
"http://example.org/Joni_Mitchell"
"http://example.org/Frank_Sinatra"
"http://example.org/The_Clash"
"http://example.org/The_Rolling_Stones"
"http://example.org/Neil_Young_&_Crazy_Horse"
"http://example.org/The_Cars"
"http://example.org/The_Smiths"
"http://example

In [205]:
#return all the entities of type album
sparql.setQuery("prefix ex: <http://example.org/> select distinct *  where {?x  a ex:Album }")
sparql.setReturnFormat("csv")
results = sparql.query()
results = results.convert()
print(results.decode())

"x"
"http://example.org/(What's_the_Story)_Morning_Glory?"
"http://example.org/Otis_Blue:_Otis_Redding_Sings_Soul"
"http://example.org/Let_It_Bleed"
"http://example.org/Illmatic"
"http://example.org/More_Songs_About_Buildings_and_Food"
"http://example.org/Astral_Weeks"
"http://example.org/Exile_in_Guyville"
"http://example.org/Blue_Lines"
"http://example.org/Master_of_Puppets"
"http://example.org/The_Velvet_Underground_&_Nico"
"http://example.org/Layla_and_Other_Assorted_Love_Songs"
"http://example.org/Paid_in_Full"
"http://example.org/Blood_Sugar_Sex_Magik"
"http://example.org/Highway_to_Hell"
"http://example.org/Automatic_for_the_People"
"http://example.org/Paul_Simon"
"http://example.org/Help!"
"http://example.org/OK_Computer"
"http://example.org/Younger_Than_Yesterday"
"http://example.org/The_Velvet_Rope"
"http://example.org/Wheels_of_Fire"
"http://example.org/Double_Nickels_on_the_Dime"
"http://example.org/Sgt._Pepper's_Lonely_Hearts_Club_Band"
"http://example.org/Tumbleweed_Conne

###### OWL inferences.

OWL , or ontology web lenguage , is a Semantic Web language for expressing ontologies designed so that its
semantics can be defined via a translation into a Description Logic (DL)1.
An ontology is a set of precise descriptive statements about some part of the
world (usually referred to as the domain of interest).
It's composed by a terminological and an assertional box.

Using OWL i can add some inferences to my knowledge, i have decide to add:
- All the triple using the owl:inverse function which i have generated in the part 2 of the project

**Now add: the triple using the owl:inverse function which i have generated in the part 2 of the project**

In [206]:
#first owl inference, add all the inverse relation of the object properties
sparql.setQuery("prefix ex: <http://example.org/>  insert {?x ?i ?y} where {?y ?z  ?x. ?i owl:inverseOf ?z}")
sparql.setReturnFormat("csv")
results = sparql.query()
results = results.convert()

It'a also possible to make inference using RDFS with the following string code:
```
# owlrl.DeductiveClosure(owlrl.RDFS_Semantics).expand(rdfs_graph) # close
print(f"{len(rdfs_graph) - len(original_graph)} triples have been added")
```

### *7. Integrate my data with DBPedia's   *

Using the `ex:AlbumName` of album names and bands in the dataframes to create a relation with DBPedia entities, then and add triples about these DBPedia IRIs to your graph.
In this part I: 

- Connect all the ex:Artist with the equivalent entity in DBpedia and in Wikidata.
- Connect all the ex:Album with the equivalent entity in DBpedia 
- Connect all the ex:genre with the equivalent entity in DBpedia 

I also add information from DBPedia to your graph with a federated query, so using SPARQL insert {} where i add:
For all album:
- Add dbr:producer for each album




e.g., the album 'Rubber Soul' on DBpedia is `dbr:Rubber_Soul`, and this 'translation' may work for other albums (don't do it manually). If the translation works (i.e., if the IRI actually exists in DBPedia) and the world album is present in the page at least 10 time i can assert that the pages is related with intented interpretattion, then add triples about these DBPedia IRIs to my graph.

Idea:
The problem wich i have to limitate with this integration technique  is that for some album name the dpedia link which I create is not talking about of what I want(e.g the album) but its information correspond to something else. So to optimize the process I count the word Album or Artist in the page to be sure that the dbr.[AlbumName or Artist] is connected with my intended interpretation.

The problem wich i have to limitate is is that for the album i generate some existence dpedia link but that doesn't correspond to my album. So to add :
- Connection between all the ex:Artist with the equivalent entity in DBpedia.
- Connection between all the ex:Album with the equivalent entity in DBpedia. 

**ex:Artist owl:sameAs dbr:Artist**

To do it i use the Artist column and the dbr:Artist. If the URL exist add the triple `ex:Artist owl:sameAs dbr:Artist`. To to it as before i modified the string Artist to obtain a correct sintax for the URL. To find the existence of the website i will import urllib.
To be sure i count the number of artist word in the website and if this number is greater then 1 i supposed that the url rapresents the identity which i believe.
This is the connection between all the ex:Artist with the equivalent entity in DBpedia i will iterate the columne and if it exist the dbr:[Artist] url and then i generate the triple.

In [207]:
#connect all the artist entity with owl:sameAs with the entities in dbpedia
from urllib.request import urlopen
from urllib.error import *
import requests
dbr = Namespace('http://dbpedia.org/resource/')
rdf_graph.bind('dbr', dbr)
#connect all the album entity with owl:sameAs with the entities in dbpedia
print('iniziato caricamento')
for inded, row in df.iterrows():
    Artist=row['Artist']
    Artist = Artist.replace(u'\xa0', u' ')
    Artist=Artist.replace(' ','_')
    
    
    
    try:
        html = urlopen(URIRef(dbr+str(Artist)))
     

    except HTTPError as e:
        print("HTTP error", e)
     
    except URLError as e:
        print("Pagina non esistente", e)
 
    else:
        x=(requests.get(URIRef(dbr+str(Artist))).text.count('Artist'))
        if x>=1:
            
            print(URIRef(ex+Artist), OWL.sameAs, URIRef(dbr+Artist))
            print(x)
            rdf_graph.add((URIRef(ex+str(Artist)), (OWL.sameAs) , URIRef(dbr+str(Artist))))
            print('è andata')
print('finito caricamento')

iniziato caricamento
http://example.org/The_Beatles http://www.w3.org/2002/07/owl#sameAs http://dbpedia.org/resource/The_Beatles
97
è andata
http://example.org/The_Beach_Boys http://www.w3.org/2002/07/owl#sameAs http://dbpedia.org/resource/The_Beach_Boys
79
è andata
http://example.org/The_Beatles http://www.w3.org/2002/07/owl#sameAs http://dbpedia.org/resource/The_Beatles
97
è andata
http://example.org/Bob_Dylan http://www.w3.org/2002/07/owl#sameAs http://dbpedia.org/resource/Bob_Dylan
120
è andata
http://example.org/The_Beatles http://www.w3.org/2002/07/owl#sameAs http://dbpedia.org/resource/The_Beatles
97
è andata
http://example.org/Marvin_Gaye http://www.w3.org/2002/07/owl#sameAs http://dbpedia.org/resource/Marvin_Gaye
56
è andata
http://example.org/The_Rolling_Stones http://www.w3.org/2002/07/owl#sameAs http://dbpedia.org/resource/The_Rolling_Stones
108
è andata
http://example.org/The_Clash http://www.w3.org/2002/07/owl#sameAs http://dbpedia.org/resource/The_Clash
55
è andata
http:

**ex:Album owl:sameAs dbr:Album**

To do it i use the Album column and the dbr:Album. As before i will verify If the URL exist add then i will add the triple `ex:Album owl:sameAs dbr:Album`. To to it as before i modified the string album to obtain a correct sintax for the URL. To find the existence of the website i will import urllib.
To be sure i count the number of 'album' word in the website and if this number is greater then 10 i supposed that the url rapresents the identity which i believe.


In [208]:
#connect all the album entity with owl:sameAs with the entities in dbpedia
print('iniziato caricamento')
import re


dbr = Namespace('http://dbpedia.org/resource/')
rdf_graph.bind('dbr', dbr)
for inded, row in df.iterrows():
    Album=row['Album']
    Album=Album.replace(u'\xa0', u' ')
    Album=Album.replace(u'\xe2', u' ')
    AlbumTitle=Album.replace(' ','_')
    AlbumTitle=AlbumTitle.replace('"','')
    try:
        html = urlopen(URIRef(dbr+str(AlbumTitle)))
        page=urlopen(URIRef(dbr+str(AlbumTitle))).read()
        
    except HTTPError as e:
        print("HTTP error", e)
    except URLError as e:
        print("Opps ! Page not found!", e)
    else:
        x=(requests.get(URIRef(dbr+str(AlbumTitle))).text.count('Album'))
        if x>=10:
            print(URIRef(ex+AlbumTitle), OWL.sameAs, URIRef(dbr+AlbumTitle))
            print(x)
            rdf_graph.add(((URIRef(dbr+AlbumTitle)), (OWL.sameAs), (URIRef(ex+AlbumTitle))))
print('finito caricamento')

iniziato caricamento
http://example.org/Sgt._Pepper's_Lonely_Hearts_Club_Band http://www.w3.org/2002/07/owl#sameAs http://dbpedia.org/resource/Sgt._Pepper's_Lonely_Hearts_Club_Band
163
http://example.org/Pet_Sounds http://www.w3.org/2002/07/owl#sameAs http://dbpedia.org/resource/Pet_Sounds
154
http://example.org/Highway_61_Revisited http://www.w3.org/2002/07/owl#sameAs http://dbpedia.org/resource/Highway_61_Revisited
120
http://example.org/Rubber_Soul http://www.w3.org/2002/07/owl#sameAs http://dbpedia.org/resource/Rubber_Soul
134
http://example.org/Exile_on_Main_St. http://www.w3.org/2002/07/owl#sameAs http://dbpedia.org/resource/Exile_on_Main_St.
96
http://example.org/London_Calling http://www.w3.org/2002/07/owl#sameAs http://dbpedia.org/resource/London_Calling
105
http://example.org/Blonde_on_Blonde http://www.w3.org/2002/07/owl#sameAs http://dbpedia.org/resource/Blonde_on_Blonde
101
http://example.org/Kind_of_Blue http://www.w3.org/2002/07/owl#sameAs http://dbpedia.org/resource/Kin

In [210]:
#using the rdf triple to add new data on sparql
for s,p,o in rdf_graph:
    sparql.setQuery(f"INSERT DATA {{{s.n3()} {p.n3()} {o.n3()}.}}")
    sparql.query()

Now i will insert the following triple:
- The name of the artist thanks rdfs:Label
- The producer of all the album, To do it i have a principal possible method:

1) Use the federated Queries-->i will write a possible code for it


#### Federated queries

Federated queries are queries that are distributed over different SPARQL endpoints. 

With it we can retrieve RDF data from multiple SPARQL endpoints such as DBPedia and Wikidata and integrate it with the data on your SPARQL endpoint.

The federated queries have the following structures:


```
    insert {
        graph <your:graph> { # what you want to insert } }
    where {
        graph <your:graph> { # data from your graph }
        service <https://dbpedia.org/sparql> { # data from dbpedia }
    }
```

This method consist in the create another SPARQLWrapper instance without default graph in order to run federated queries, i need to cancel the default graph.

In [218]:
sparql = SPARQLWrapper("http://localhost:8890/sparql")
sparql.method = 'POST'


Define a couple functions that will run these queries:

In [219]:
def run_query(query, sparql=sparql, method='GET', format='csv'):
 
    sparql.setQuery(query)
    
    sparql.setReturnFormat(format)
    
    sparql.method = method
    
    try : 
    
        return sparql.query()
        
    except Exception as e:
    
        print(e)
        
        print("query failed")

def run_count(sparql=sparql):

    results = run_query(count, sparql, format='json').convert()
    
    return int(results['results']['bindings'][0]['count']['value'])

# sparql = SPARQLWrapper("http://80.211.177.54:8890/sparql/")
sparql = SPARQLWrapper("http://localhost:8890/sparql")
# # setting a default graph does not work with
# # federated queries for whatever reason
#sparql.addDefaultGraph("ex:sw")
sparql.method = 'POST'


# IMPORTANT
#### AFTER ALL THE GRAPH CONTRUCTION , SO EXACTLY NOW DO THIS PROCEDURE 
To use the federate query i need to come back on gitbash and:
1) Goes to the directory of virtuoso

2) write $ ./virtuoso.sh isql

3) Then paste the following after SQL>:
> - grant SPARQL_LOAD_SERVICE_DATA to "SPARQL";
>- GRANT SELECT ON "DB"."DBA"."SPARQL_SINV_2" TO "SPARQL";
>- GRANT EXECUTE ON "DB"."DBA"."SPARQL_SINV_IMP" TO "SPARQL";
>- GRANT INSERT ON "DB"."DBA"."SPARQL_SINV_2" TO "SPARQL";
>- grant execute on "DB.DBA.EXEC_AS" to "SPARQL";



The idea is to take the information from dpedia to add the information , here i add the name using the rdfs label contained in dbpedia. Before to use the following queries to :
-Find the name of the artist:

In [227]:
sparql.setQuery( """

 

    prefix ex: <http://example.org/>
    
    prefix dbp: <http://dbpedia.org/property/>
    
select ?x ?name 
        
    where {
    
        graph <ex:509653> { ?x owl:sameAs ?artist; a ex:Artist}
        
        service <https://dbpedia.org/sparql> { ?artist rdfs:label ?name}
    }
 
    
""")
sparql.setReturnFormat("csv")
results = sparql.query()
results = results.convert()
print (results.decode())


"x","name"
"http://example.org/Paul_Simon","Paul Simon"
"http://example.org/Paul_Simon","بول سايمون"
"http://example.org/Paul_Simon","Paul Simon"
"http://example.org/Paul_Simon","Paul Simon"
"http://example.org/Paul_Simon","Paul Simon"
"http://example.org/Paul_Simon","Πολ Σάιμον"
"http://example.org/Paul_Simon","Paul Simon"
"http://example.org/Paul_Simon","Paul Simon"
"http://example.org/Paul_Simon","Paul Simon"
"http://example.org/Paul_Simon","Paul Simon"
"http://example.org/Paul_Simon","Paul Simon"
"http://example.org/Paul_Simon","Paul Simon (chanteur)"
"http://example.org/Paul_Simon","Paul Simon"
"http://example.org/Paul_Simon","폴 사이먼"
"http://example.org/Paul_Simon","ポール・サイモン"
"http://example.org/Paul_Simon","Paul Simon (artiest)"
"http://example.org/Paul_Simon","Paul Simon"
"http://example.org/Paul_Simon","Саймон, Пол"
"http://example.org/Paul_Simon","Paul Simon"
"http://example.org/Paul_Simon","Пол Саймон"
"http://example.org/Paul_Simon","Paul Simon"
"http://example.org/Paul_Simo

In [224]:
sparql.setQuery("""

 

    prefix ex: <http://example.org/>
    
    prefix dbp: <http://dbpedia.org/property/>
    
insert{ graph <ex:509653> {?x rdfs:label ?name }}
where {
        graph <ex:509653> { ?x owl:sameAs ?artist; a ex:Artist}
        
        service <https://dbpedia.org/sparql> { ?artist rdfs:label ?name}
    }
    
""")
sparql.setReturnFormat("csv")
results = sparql.query()
results = results.convert()
print(results.decode())
#print(results.convert().toxml())



EndPointInternalError: EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'Virtuoso 42000 Error SR186:SECURITY: No permission to execute procedure DB.DBA.SPARQL_INSERT_DICT_CONTENT with user ID 108, group ID 108\n\nSPARQL query:\ndefine sql:big-data-const 0\n\n\n \n\n    prefix ex: <http://example.org/>\n    \n    prefix dbp: <http://dbpedia.org/property/>\n    \ninsert{ graph <ex:509653> {?x rdfs:label ?name }}\nwhere {\n        graph <ex:509653> { ?x owl:sameAs ?artist; a ex:Artist}\n        \n        service <https://dbpedia.org/sparql> { ?artist rdfs:label ?name}\n    }\n    \n'

In [ ]:
sparql.setQuery("""

    prefix ex: <http://example.org/>
    
    prefix dbp: <http://dbpedia.org/property/>
    
    insert {
    
        graph <ex:509653> { ?x dbp:producer ?prod } }
        
    where {
    
        graph <ex:509653> { ?x owl:sameAs ?al; a ex:Album}
        
        service <https://dbpedia.org/sparql> { ?al dbp:producer ?prod }
    }
""")
sparql.setReturnFormat("csv")
results = sparql.query()
results = results.convert()
print(results.decode())

# THE END

In [209]:
#Print Graph
print(rdf_graph.serialize())

@prefix dbr: <http://dbpedia.org/resource/> .
@prefix ex: <http://example.org/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:Album a rdfs:Class .

ex:Artist a rdfs:Class .

ex:Blues a rdfs:Class ;
    rdfs:subClassOf ex:Genre .

ex:Country a rdfs:Class ;
    rdfs:subClassOf ex:Country,
        ex:Funk,
        ex:Genre,
        ex:Soul .

ex:Electronic a rdfs:Class ;
    rdfs:subClassOf ex:Genre .

ex:Folk a rdfs:Class ;
    rdfs:subClassOf ex:Country,
        ex:Genre .

ex:Funk a rdfs:Class ;
    rdfs:subClassOf ex:Country,
        ex:Funk,
        ex:Genre,
        ex:Screen,
        ex:Soul,
        ex:Stage .

ex:Genre a rdfs:Class .

ex:HipHop a rdfs:Class ;
    rdfs:subClassOf ex:Funk,
        ex:Genre,
        ex:Soul .

ex:Jazz a rdfs:Class ;
    rdfs:subClassOf ex:Genre .

ex:Latin a rdfs:Class ;
    rdfs:subClassOf ex:Genre .

ex:Pop a rdfs:Class ;
    rdfs:sub